In [2]:
import gym
import numpy as np
import copy, random, os, subprocess, cv2
from tensorflow.keras import layers, models

/opt/conda/lib/python3.7/site-packages/flatbuffers/compat.py:19: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
/opt/conda/lib/python3.7/site-packages/keras_preprocessing/image/utils.py:23: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  'nearest': pil_image.NEAREST,
/opt/conda/lib/python3.7/site-packages/keras_preprocessing/image/utils.py:24: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  'bilinear': pil_image.BILINEAR,
/opt/conda/lib/python3.7/site-packages/keras_preprocessing/image/utils.py:25: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BICUBIC instead.
  'bicubic': pil_image.BICUBIC,
/opt/conda/lib/python3.7/site-packages/keras_preprocessing/image/utils.py:28: Depreca

In [3]:
MODEL_NAME = 'model05'
BUCKET = 'gs://etsuji-car-racing-v2'
os.environ['BUCKET'] = BUCKET
!gsutil mb -c regional -l us-west1 $BUCKET
!gsutil ls $BUCKET

Creating gs://etsuji-car-racing-v2/...
ServiceException: 409 A Cloud Storage bucket named 'etsuji-car-racing-v2' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.
gs://etsuji-car-racing-v2/model04/


In [4]:
# Base model
class QValue:
    def __init__(self):
        self.model = self.build_model()

    def build_model(self):
        cnn_input = layers.Input(shape=(48, 48, 3), name='cnn_input')
        cnn1 = layers.Conv2D(32, (5, 5), padding='same',
                         use_bias=True, activation='relu',
                        name='cnn1')(cnn_input)
        pool1 = layers.MaxPooling2D((2, 2), name='pool1')(cnn1)
        cnn2 = layers.Conv2D(64, (5, 5), padding='same',
                         use_bias=True, activation='relu',
                        name='cnn2')(pool1)
        pool2 = layers.MaxPooling2D((2, 2), name='pool2')(cnn2)

        cnn_flatten = layers.Flatten(name='flatten')(pool2)
        action_input = layers.Input(shape=(5,), name='action_input')
        combined = layers.concatenate([cnn_flatten, action_input], name='concat')
        hidden1 = layers.Dense(1024, activation='relu', name='dense1')(combined)
        hidden2 = layers.Dense(512, activation='relu', name='dense2')(hidden1)
        q_value = layers.Dense(1, name='output')(hidden2)

        model = models.Model(inputs=[cnn_input, action_input], outputs=q_value)
        model.compile(loss='mse')
        return model

    def get_action(self, state):
        states = []
        actions = []
        for a in range(5):
            states.append(np.array(state))
            action_onehot = np.zeros(5)
            action_onehot[a] = 1
            actions.append(action_onehot)
  
        q_values = self.model.predict([np.array(states), np.array(actions)])
        optimal_action = np.argmax(q_values)
        return optimal_action, q_values[optimal_action][0]

In [15]:
def join_frames(o0, o1, o2):
    gray_image0 = cv2.cvtColor(cv2.resize(o0, (48, 48)), cv2.COLOR_RGB2GRAY)
    gray_image1 = cv2.cvtColor(cv2.resize(o1, (48, 48)), cv2.COLOR_RGB2GRAY)
    gray_image2 = cv2.cvtColor(cv2.resize(o2, (48, 48)), cv2.COLOR_RGB2GRAY)
    
    return np.array(
        [gray_image0.transpose(),
         gray_image1.transpose(),
         gray_image2.transpose()]).transpose()

In [11]:
def get_episode(environ, q_value, epsilon):
    episode = []
    o0 = environ.reset()
    o1 = copy.deepcopy(o0)
    o2 = copy.deepcopy(o0)
    total_r = 0

    if epsilon > 0:
        keep_count = 3
    else:
        keep_count = 1

    c = 0
    while True:
        if c % keep_count == 0: # Get new action
            if np.random.random() < epsilon:
                a = np.random.randint(5)
            else:
                a, _ = q_value.get_action(join_frames(o0, o1, o2))
        c += 1
        o_new, r, done, inf = environ.step(a)                
        total_r += r

        # Terminate episode when total reward becomes negative
        if total_r < 0:
            done = 1

        if done:
            # Terminal state is to achieve more than 990 or get out of the field.
            if total_r > 990 or r < -99:
                episode.append((join_frames(o0, o1, o2), a, r, None))
            break
        else:
            episode.append((join_frames(o0, o1, o2), a, r, join_frames(o1, o2, o_new)))
        o0, o1, o2 = o1, o2, o_new

    print('epsilon={}, episode length={}, total rewards={}'.format(epsilon, len(episode), total_r))
    return episode, total_r

In [8]:
def train(environ, q_value, epsilon, checkpoint=0):
    if checkpoint > 0:
        filename = 'car-racing-v2-{}-{}.hd5'.format(checkpoint, MODEL_NAME)
        subprocess.run(['gsutil', 'cp', '{}/{}/{}'.format(BUCKET, MODEL_NAME, filename), './'])
        print('load model {}'.format(filename))
        q_value.model = models.load_model(filename)
        os.remove(filename)

    experience = []
    good_experience = []
    best_r = [-100, -100, -100]

    for n in range(checkpoint + 1, checkpoint + 1000):
        print('iteration {}'.format(n))

        total_len = 0
        if n % 3 == 0:
            print('Testing the current performance...')
            episode, total_r = get_episode(environ, q_value, epsilon=0)
            with open('result.txt', 'a') as f:
                f.write('{},{},{},{}\n'.format(n, epsilon, len(episode), total_r))
            filename = 'car-racing-v2-{}-{}.hd5'.format(n, MODEL_NAME)
            q_value.model.save(filename, save_format='h5')
            subprocess.run(['gsutil', '-m', 'cp',
                            '{}'.format(filename), '{}/{}/'.format(BUCKET, MODEL_NAME)])
            os.remove(filename)
            experience += episode
            total_len += len(episode)

        while total_len < 500:
            episode, total_r = get_episode(environ, q_value, epsilon)
            total_len += len(episode)
            experience += episode

            # Keep the top 3 episodes
            if total_r > min(best_r):
                best_r = best_r[1:] + [total_r]
                good_experience += episode
                if len(good_experience) > 999 * 3:
                    good_experience = good_experience[-999 * 3:]

            
        if len(experience) > 999 * 5: # remember last 5 episodes
            experience = experience[-999 * 5:]

        epsilon = (epsilon - 0.2) * 0.99 + 0.2

        print('Training the model...')
        # Use latest episode + past episodes (sampling) + top 3 episode (sampling)
        latest_experience = experience[-total_len:]
        past_experience = experience[:-total_len]
        examples = latest_experience + \
            random.sample(past_experience, min(len(past_experience), 999)) + \
            random.sample(good_experience, min(len(good_experience), 999))
        
        # Show some statistics
        print('experience length={}'.format(len(experience)))
        print('number of examples={}'.format(len(examples)))
        print('best total reward = ', best_r)
        np.random.shuffle(examples)
                        
        states, actions, labels = [], [], []
        for state, a, r, state_new in examples:
            states.append(np.array(state))

            action_onehot = np.zeros(5)
            action_onehot[a] = 1
            actions.append(action_onehot)
            
            if state_new is None:   # Terminal state
                q_new = 0
            else:
                _, q_new = q_value.get_action(state_new)
            labels.append(np.array(r + q_new))

        hist = q_value.model.fit(
            [np.array(states), np.array(actions)], np.array(labels),
            batch_size=50, epochs=10, verbose=0)
        print('loss = {}'.format(hist.history['loss']))

In [9]:
env = gym.make("CarRacing-v2", continuous=False)
q_value = QValue()
q_value.model.summary()

/opt/conda/lib/python3.7/site-packages/gym/core.py:330: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  "Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future."
/opt/conda/lib/python3.7/site-packages/gym/wrappers/step_api_compatibility.py:40: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  "Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future."


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 cnn_input (InputLayer)         [(None, 48, 48, 3)]  0           []                               
                                                                                                  
 cnn1 (Conv2D)                  (None, 48, 48, 32)   2432        ['cnn_input[0][0]']              
                                                                                                  
 pool1 (MaxPooling2D)           (None, 24, 24, 32)   0           ['cnn1[0][0]']                   
                                                                                                  
 cnn2 (Conv2D)                  (None, 24, 24, 64)   51264       ['pool1[0][0]']                  
                                                                                              

2022-07-31 04:28:35.903310: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-31 04:28:36.019379: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-31 04:28:36.020346: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-31 04:28:36.023118: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [ ]:
train(env, q_value, epsilon=1.0, checkpoint=0)

iteration 1
epsilon=1.0, episode length=388, total rewards=-0.030742049469842686
epsilon=1.0, episode length=103, total rewards=-0.019377162629740513
epsilon=1.0, episode length=500, total rewards=-0.09999999999991549
Training the model...
experience length=991
number of examples=1982
best total reward =  [-0.030742049469842686, -0.019377162629740513, -0.09999999999991549]


2022-07-31 04:32:17.516918: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8200


loss = [497956.53125, 73.32007598876953, 77.51648712158203, 92.5329818725586, 120.36689758300781, 170.72515869140625, 140.1612548828125, 72.56980895996094, 43.81856155395508, 40.56475830078125]
iteration 2
epsilon=0.992, episode length=190, total rewards=-0.08859315589349515
epsilon=0.992, episode length=325, total rewards=-0.02671009771977781
Training the model...
experience length=1506
number of examples=2505
best total reward =  [-0.09999999999991549, -0.08859315589349515, -0.02671009771977781]
loss = [21.975669860839844, 20.485933303833008, 17.40968132019043, 18.9661922454834, 14.760263442993164, 14.251534461975098, 11.81113052368164, 11.811808586120605, 10.329019546508789, 10.2379732131958]
iteration 3
Testing the current performance...
epsilon=0, episode length=71, total rewards=-0.05714285714284828


Copying file://car-racing-v2-3-model05.hd5 [Content-Type=application/octet-stream]...
\ [1/1 files][ 76.5 MiB/ 76.5 MiB] 100% Done                                    
Operation completed over 1 objects/76.5 MiB.                                     


epsilon=0.9840800000000001, episode length=123, total rewards=-0.05432098765429427
epsilon=0.9840800000000001, episode length=203, total rewards=-0.06101694915252076
epsilon=0.9840800000000001, episode length=132, total rewards=-0.0986798679867793
Training the model...
experience length=2035
number of examples=2527
best total reward =  [-0.02671009771977781, -0.05432098765429427, -0.06101694915252076]
loss = [7.227925777435303, 7.030678749084473, 6.519842624664307, 6.486823558807373, 6.387738227844238, 5.618881702423096, 5.925346374511719, 5.600024223327637, 5.260457992553711, 4.725531101226807]
iteration 4
epsilon=0.9762392000000002, episode length=147, total rewards=-0.09411764705880343
epsilon=0.9762392000000002, episode length=176, total rewards=-0.09436619718307004
epsilon=0.9762392000000002, episode length=247, total rewards=-0.06501766784446716
Training the model...
experience length=2605
number of examples=2568
best total reward =  [-0.02671009771977781, -0.05432098765429427, -

...

Training the model...
experience length=4995
number of examples=2997
best total reward =  [597.761194029839, 790.1515151515008, 750.7462686567039]
loss = [22.41435432434082, 18.635210037231445, 16.733139038085938, 15.326617240905762, 14.384572982788086, 11.8331937789917, 13.066092491149902, 11.938275337219238, 10.754467964172363, 11.012728691101074]
iteration 514
epsilon=0.20461258960078169, episode length=999, total rewards=413.79310344826223
Training the model...
experience length=4995
number of examples=2997
best total reward =  [597.761194029839, 790.1515151515008, 750.7462686567039]
loss = [18.747873306274414, 14.700261116027832, 13.032047271728516, 11.159309387207031, 10.847208976745605, 10.07747745513916, 10.065681457519531, 8.705581665039062, 11.911310195922852, 8.376543998718262]
iteration 515
epsilon=0.20456646370477388, episode length=825, total rewards=168.15350553504788
Training the model...
experience length=4995
number of examples=2823
best total reward =  [597.761194029

Copying file://car-racing-v2-516-model05.hd5 [Content-Type=application/octet-stream]...
- [1/1 files][ 76.5 MiB/ 76.5 MiB] 100% Done                                    
Operation completed over 1 objects/76.5 MiB.                                     


Training the model...
experience length=4995
number of examples=2997
best total reward =  [597.761194029839, 790.1515151515008, 750.7462686567039]
loss = [21.683401107788086, 15.528409957885742, 14.292512893676758, 15.391136169433594, 10.822954177856445, 12.449285507202148, 11.059289932250977, 11.539695739746094, 9.602621078491211, 10.255187034606934]
iteration 517
epsilon=0.20447559107704888, episode length=999, total rewards=534.1463414634043
Training the model...
experience length=4995
number of examples=2997
best total reward =  [597.761194029839, 790.1515151515008, 750.7462686567039]
loss = [35.64896011352539, 32.04753112792969, 25.163307189941406, 23.157428741455078, 19.196155548095703, 18.711183547973633, 17.066938400268555, 30.190265655517578, 13.191025733947754, 13.256834983825684]
iteration 518
epsilon=0.20443083516627839, episode length=999, total rewards=125.98870056497572
Training the model...
experience length=4995
number of examples=2997
best total reward =  [597.7611940

Copying file://car-racing-v2-519-model05.hd5 [Content-Type=application/octet-stream]...
- [1/1 files][ 76.5 MiB/ 76.5 MiB] 100% Done                                    
Operation completed over 1 objects/76.5 MiB.                                     


Training the model...
experience length=4995
number of examples=2997
best total reward =  [597.761194029839, 790.1515151515008, 750.7462686567039]
loss = [27.88681983947754, 15.317206382751465, 13.458620071411133, 13.091867446899414, 11.23193645477295, 11.433625221252441, 12.933049201965332, 10.367365837097168, 45.96946334838867, 7.707971096038818]
iteration 520
epsilon=0.20434266154646946, episode length=999, total rewards=675.0865051903039
Training the model...
experience length=4995
number of examples=2997
best total reward =  [790.1515151515008, 750.7462686567039, 675.0865051903039]
loss = [16.926267623901367, 14.409167289733887, 12.227670669555664, 11.827457427978516, 11.072009086608887, 9.856014251708984, 9.720939636230469, 10.744915962219238, 9.62158203125, 10.041657447814941]
iteration 521
epsilon=0.20429923493100477, episode length=999, total rewards=247.22222222222726
Training the model...
experience length=4995
number of examples=2997
best total reward =  [790.1515151515008,